In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import io
import re

import matplotlib.pyplot as plt
import numpy as np
import requests

import deepxde as dde
from deepxde.backend import tf

import plotly.graph_objects as go
import pandas as pd

import plotly.express as px
from plotly.subplots import make_subplots

Using TensorFlow 2 backend.

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
def pde(xy, u):
    dudx = dde.grad.jacobian(u, xy, i=0, j=0)
    dudy = dde.grad.jacobian(u, xy, i=0, j=1)
    du2dxx = dde.grad.hessian(u, xy, i=0, j=0)
    du2dxy = dde.grad.hessian(u, xy, i=0, j=1)
    du2dyx = dde.grad.hessian(u, xy, i=1, j=0)
    du2dyy = dde.grad.hessian(u, xy, i=1, j=1)
    return u+dudx+dudy+du2dxx+du2dxy+du2dyx+du2dyy

In [3]:
t = np.linspace(0, 2*np.pi, 200)
bc_x = np.cos(t)
bc_y = np.sin(t)
bc_points = np.vstack([bc_x,bc_y]).T
bc_values_cos_list = [np.expand_dims(np.cos(i*t),-1) for i in range(6)]
bc_values_sin_list = [np.expand_dims(np.sin(i*t),-1) for i in range(6)]

In [4]:
geom = dde.geometry.Rectangle([-1, -1], [1, 1])
bc_cos_list = [dde.PointSetBC(bc_points, bc_values_cos_list[i]) for i in range(6)]
bc_sin_list = [dde.PointSetBC(bc_points, bc_values_sin_list[i]) for i in range(6)]

In [5]:
data_cos_list = [dde.data.PDE(geom, pde, bc_cos_list[i], num_domain=1600 , num_boundary=400, num_test=1600 , train_distribution = "uniform") for i in range(6)]
data_sin_list = [dde.data.PDE(geom, pde, bc_sin_list[i], num_domain=1600 , num_boundary=400, num_test=1600 , train_distribution = "uniform") for i in range(6)]

In [6]:
net_cos_list = [dde.maps.FNN([2] + [50] * 4 + [1], "tanh", "Glorot uniform") for i in range(6)]
net_sin_list = [dde.maps.FNN([2] + [50] * 4 + [1], "tanh", "Glorot uniform") for i in range(6)]

In [7]:
model_cos_list = [dde.Model(data_cos_list[i], net_cos_list[i]) for i in range(6)]
model_sin_list = [dde.Model(data_sin_list[i], net_sin_list[i]) for i in range(6)]

In [8]:
for model_cos in model_cos_list:
    model_cos.compile("adam", lr=0.0002)
    model_cos.train(epochs=5000)
    model_cos.compile("L-BFGS-B")
    model_cos.train()

Compiling model...
Building feed-forward neural network...
'build' took 0.062837 s



c:\programdata\anaconda3\envs\deepxde\lib\site-packages\tensorflow\python\keras\legacy_tf_layers\core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
c:\programdata\anaconda3\envs\deepxde\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


'compile' took 0.769568 s

Initializing variables...
Training model...

Step      Train loss              Test loss               Test metric
0         [1.83e-02, 1.00e+00]    [1.88e-02, 0.00e+00]    []  
1000      [6.68e-04, 6.27e-04]    [5.38e-04, 0.00e+00]    []  
2000      [1.16e-04, 5.59e-05]    [1.12e-04, 0.00e+00]    []  
3000      [2.45e-05, 1.75e-05]    [2.04e-05, 0.00e+00]    []  
4000      [1.32e-05, 1.10e-05]    [1.08e-05, 0.00e+00]    []  
5000      [1.39e-04, 3.52e-05]    [1.39e-04, 0.00e+00]    []  

Best model at step 4000:
  train loss: 2.42e-05
  test loss: 1.08e-05
  test metric: []

'train' took 88.562566 s

Compiling model...
'compile' took 0.281328 s

Training model...

Step      Train loss              Test loss               Test metric
5000      [1.39e-04, 3.52e-05]    [1.39e-04, 0.00e+00]    []  
INFO:tensorflow:Optimization terminated with:
  Message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  Objective function value: 0.000001
  Number of iterations:

In [9]:
for model_sin in model_sin_list:
    model_sin.compile("adam", lr=0.0002)
    model_sin.train(epochs=5000)
    model_sin.compile("L-BFGS-B")
    model_sin.train()

Compiling model...
Building feed-forward neural network...
'build' took 0.059695 s

'compile' took 0.743108 s

Initializing variables...
Training model...

Step      Train loss              Test loss               Test metric
0         [7.00e-03, 6.21e-03]    [7.10e-03, 0.00e+00]    []  
1000      [6.76e-07, 2.39e-09]    [5.51e-07, 0.00e+00]    []  
2000      [4.29e-07, 6.40e-09]    [3.52e-07, 0.00e+00]    []  
3000      [2.54e-07, 4.86e-09]    [2.08e-07, 0.00e+00]    []  
4000      [1.20e-06, 2.83e-07]    [1.18e-06, 0.00e+00]    []  
5000      [7.13e-06, 1.78e-06]    [7.22e-06, 0.00e+00]    []  

Best model at step 3000:
  train loss: 2.59e-07
  test loss: 2.08e-07
  test metric: []

'train' took 110.859384 s

Compiling model...
'compile' took 0.569624 s

Training model...

Step      Train loss              Test loss               Test metric
5000      [7.13e-06, 1.78e-06]    [7.22e-06, 0.00e+00]    []  
INFO:tensorflow:Optimization terminated with:
  Message: CONVERGENCE: NORM_OF_PRO

In [145]:
x, y = np.linspace(-1, 1, 21), np.linspace(-1, 1, 21)
x_mesh,y_mesh = np.meshgrid(x, y)

In [11]:
cos_pred_list = [model_cos_list[i].predict(np.vstack([x_mesh.flatten(),y_mesh.flatten()]).T).view().reshape(x_mesh.shape) for i in range(6)]
sin_pred_list = [model_sin_list[i].predict(np.vstack([x_mesh.flatten(),y_mesh.flatten()]).T).view().reshape(x_mesh.shape) for i in range(6)]

Predicting...
'predict' took 1.030819 s

Predicting...
'predict' took 1.004271 s

Predicting...
'predict' took 0.991258 s

Predicting...
'predict' took 1.003349 s

Predicting...
'predict' took 0.979374 s

Predicting...
'predict' took 0.963515 s

Predicting...
'predict' took 0.952645 s

Predicting...
'predict' took 0.940250 s

Predicting...
'predict' took 0.925072 s

Predicting...
'predict' took 0.905511 s

Predicting...
'predict' took 0.891557 s

Predicting...
'predict' took 0.171268 s



In [28]:
bc_cos_pred_list = [model_cos_list[i].predict(np.vstack([bc_x,bc_y]).T) for i in range(6)]
bc_sin_pred_list = [model_sin_list[i].predict(np.vstack([bc_x,bc_y]).T) for i in range(6)]

Predicting...
'predict' took 0.000998 s

Predicting...
'predict' took 0.000998 s

Predicting...
'predict' took 0.000997 s

Predicting...
'predict' took 0.000998 s

Predicting...
'predict' took 0.000997 s

Predicting...
'predict' took 0.000997 s

Predicting...
'predict' took 0.000000 s

Predicting...
'predict' took 0.000000 s

Predicting...
'predict' took 0.000997 s

Predicting...
'predict' took 0.000997 s

Predicting...
'predict' took 0.000997 s

Predicting...
'predict' took 0.000997 s



## Test the decomposition

In [40]:
bc_values = np.expand_dims(-(t-np.pi+np.sin(t-np.pi))*np.exp(-(t-np.pi)**2),-1)

In [112]:
period = 2*np.pi

def an(x,fx,dx,P,n):
    an = 2/P*np.sum(fx*np.cos(2*np.pi*n*x/P))*dx
    return an

def bn(x,fx,dx,P,n):
    bn = 2/P*np.sum(fx*np.sin(2*np.pi*n*x/P))*dx
    return bn

def SN(x,fx,dx,P,Nh):
    an_list = [an(x,fx,dx,P,i) for i in range(Nh)]
    bn_list = [bn(x,fx,dx,P,i) for i in range(Nh)]
    sn = np.zeros_like(fx) + an_list[0]/2
    for i in range(1,Nh):
        sn = sn + an_list[i]*np.cos(2*np.pi*i*x/P) + bn_list[i]*np.sin(2*np.pi*i*x/P)
    return sn

def cn(n):
    c = bc_values[:,0]*np.exp(-1j*2*n*np.pi*t/period)
    return c.sum()/c.size

def f(x, Nh):
    f = np.array([2*cn(i)*np.exp(1j*2*i*np.pi*x/period) for i in range(1,Nh+1)])
    return f.sum()

In [134]:
data = {'t':t,'v':bc_values[:,0],
        '1':SN(t,bc_values[:,0],2*np.pi/200,2*np.pi,2),
        '2':SN(t,bc_values[:,0],2*np.pi/200,2*np.pi,3),
        '3':SN(t,bc_values[:,0],2*np.pi/200,2*np.pi,4),
        '4':SN(t,bc_values[:,0],2*np.pi/200,2*np.pi,5),
        '5':SN(t,bc_values[:,0],2*np.pi/200,2*np.pi,6)}
  
# Create DataFrame 
df = pd.DataFrame(data) 

In [135]:
fig = go.Figure(data=[go.Scatter(x=data['t'], y=data['v']),
                      go.Scatter(x=data['t'], y=data['1']),
                      go.Scatter(x=data['t'], y=data['2']),
                      go.Scatter(x=data['t'], y=data['3']),
                      go.Scatter(x=data['t'], y=data['4']),
                      go.Scatter(x=data['t'], y=data['5'])])
fig.show()

In [136]:
cos_coef_list = [an(t,bc_values[:,0],2*np.pi/200,2*np.pi,i) for i in range(6)]
sin_coef_list = [bn(t,bc_values[:,0],2*np.pi/200,2*np.pi,i) for i in range(6)]

In [139]:
solution_recovery = cos_pred_list[0]*cos_coef_list[0]/2
for i in [1,2,3,4,5]:
    solution_recovery = solution_recovery + cos_pred_list[i]*cos_coef_list[i] + sin_pred_list[i]*sin_coef_list[i]

In [140]:
bc_recovery = bc_cos_pred_list[0]*cos_coef_list[0]/2
for i in [1,2,3,4,5]:
    bc_recovery = bc_recovery + bc_cos_pred_list[i]*cos_coef_list[i] + bc_sin_pred_list[i]*sin_coef_list[i]

In [141]:
data = {'x':bc_x,'y':bc_y,'bc_recovery':bc_recovery[:,0]} 
  
# Create DataFrame 
df = pd.DataFrame(data) 

In [146]:
fig = go.Figure(data=[go.Scatter3d(x=data['x'], y=data['y'], z=data['bc_recovery'],
                                   marker=dict(size=4,color=data['bc_recovery'],colorscale='Viridis'),
                                   line=dict(color=data['bc_recovery'],width=2)),
                      go.Surface(z=solution_recovery, x=x, y=y)])
fig.show()

In [64]:
bc_real = dde.PointSetBC(bc_points, bc_values)
data_real = dde.data.PDE(geom, pde, bc_real, num_domain=1600 , num_boundary=400, num_test=1600 , train_distribution = "uniform")

net_real = dde.maps.FNN([2] + [50] * 4 + [1], "tanh", "Glorot uniform")

model_real = dde.Model(data_real, net_real)

In [37]:
model_real.compile("adam", lr=0.0001)
model_real.train(epochs=10000)
model_real.compile("L-BFGS-B")

Compiling model...
Building feed-forward neural network...
'build' took 0.061835 s



c:\programdata\anaconda3\envs\deepxde\lib\site-packages\tensorflow\python\keras\legacy_tf_layers\core.py:171: UserWarning:

`tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.

c:\programdata\anaconda3\envs\deepxde\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1719: UserWarning:

`layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.



'compile' took 0.955502 s

Initializing variables...
Training model...

Step      Train loss              Test loss               Test metric
0         [1.51e-01, 3.02e-01]    [1.56e-01, 0.00e+00]    []  
1000      [3.02e-03, 6.65e-03]    [2.63e-03, 0.00e+00]    []  
2000      [2.64e-04, 1.10e-03]    [2.52e-04, 0.00e+00]    []  
3000      [5.26e-05, 2.06e-04]    [4.66e-05, 0.00e+00]    []  
4000      [2.56e-05, 5.07e-05]    [2.14e-05, 0.00e+00]    []  
5000      [1.89e-05, 2.07e-05]    [1.58e-05, 0.00e+00]    []  
6000      [1.38e-05, 1.23e-05]    [1.17e-05, 0.00e+00]    []  
7000      [2.96e-05, 2.09e-05]    [2.76e-05, 0.00e+00]    []  
8000      [7.34e-06, 6.39e-06]    [6.33e-06, 0.00e+00]    []  
9000      [8.79e-06, 6.51e-06]    [8.40e-06, 0.00e+00]    []  
10000     [4.10e-06, 4.17e-06]    [3.53e-06, 0.00e+00]    []  

Best model at step 10000:
  train loss: 8.27e-06
  test loss: 3.53e-06
  test metric: []

'train' took 630.056883 s

Compiling model...
'compile' took 0.874464 s



In [38]:
real_pred = model_real.predict(np.vstack([x_mesh.flatten(),y_mesh.flatten()]).T).view().reshape(x_mesh.shape)

Predicting...
'predict' took 0.996933 s



In [65]:
fig = go.Figure(data=[go.Scatter3d(x=data['x'], y=data['y'], z=bc_values[:,0],
                                   marker=dict(size=4,color=data['bc_recovery'],colorscale='Viridis'),
                                   line=dict(color=data['bc_recovery'],width=2)),
                      go.Surface(z=real_pred, x=x, y=y)])
fig.show()